<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Mathematics Basics

**With `NumPy`**

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Prime Numbers

### Pure Python

In [ ]:
!git clone https://github.com/tpq-classes/mathematics_basics.git
import sys
sys.path.append('mathematics_basics')


In [ ]:
def is_prime(I):
    if I % 2 == 0: return False  
    for i in range(3, int(I ** 0.5) + 1, 2):  
        if I % i == 0: return False  
    return True  

In [ ]:
n = int(1e8 + 3)  
n

In [ ]:
%time is_prime(n)

In [ ]:
p1 = int(1e8 + 7)  
p1

In [ ]:
%time is_prime(p1)

In [ ]:
p2 = 100109100129162907

In [ ]:
p2.bit_length()  

In [ ]:
%time is_prime(p2)

### Numba

In [ ]:
import numba

In [ ]:
is_prime_nb = numba.jit(is_prime)

In [ ]:
%time is_prime_nb(n)  

In [ ]:
%time is_prime_nb(n) 

In [ ]:
%time is_prime_nb(p1)

In [ ]:
%time is_prime_nb(p2)  

### Cython

In [ ]:
%load_ext Cython

In [ ]:
%%cython
def is_prime_cy1(I):
    if I % 2 == 0: return False
    for i in range(3, int(I ** 0.5) + 1, 2):
        if I % i == 0: return False
    return True

In [ ]:
%timeit is_prime(p1)

In [ ]:
%timeit is_prime_cy1(p1)

In [ ]:
%%cython
def is_prime_cy2(long I):  
    cdef long i
    if I % 2 == 0: return False
    for i in range(3, int(I ** 0.5) + 1, 2):
        if I % i == 0: return False
    return True

In [ ]:
%timeit is_prime_cy2(p1)

In [ ]:
%time is_prime_cy2(p2)

In [ ]:
%time is_prime_nb(p2)

In [ ]:
%timeit is_prime_cy2(p2)

## `numexpr` Module

In [ ]:
import numpy as np

In [ ]:
import numexpr as ne
np.set_printoptions(suppress=True)

In [ ]:
N = 10000000

In [ ]:
%time a = np.linspace(100, 200, N)

In [ ]:
%time (np.sqrt(a) + np.log(a)) * np.sin(a)

In [ ]:
%timeit (np.sqrt(a) + np.log(a)) * np.sin(a)

In [ ]:
ex = '(sqrt(a) + log(a)) * sin(a)'

In [ ]:
ne.set_num_threads(1)

In [ ]:
%time ne.evaluate(ex)

In [ ]:
%timeit ne.evaluate(ex)

In [ ]:
ne.set_num_threads(8)

In [ ]:
%time ne.evaluate(ex)

In [ ]:
%timeit ne.evaluate(ex)

## Numerical Expressions

In [ ]:
def perf_comp_data(func_list, data_list, rep=3, number=1):
    from timeit import repeat
    res_list = {}
    for name in enumerate(func_list):
        stmt = name[1] + '(' + data_list[name[0]] + ')'
        setup = 'from __main__ import ' + name[1] + ', ' + data_list[name[0]]
        results = repeat(stmt=stmt, setup=setup, repeat=rep, number=number)
        res_list[name[1]] = sum(results) / rep
    res_sort = sorted(res_list.items(), key=lambda item: item[1])
    for item in res_sort:
        rel = item[1] / res_sort[0][1]
        print('function: ' + item[0] + ', av. time sec: %9.5f, ' % item[1] \
            + 'relative: %6.1f' % rel)

A typical compute intensive operation is to evaluate a **complex mathematical expression on a large array of data**.

First, let us define an **example function/numerical expression**.

In [ ]:
import math
def f(x):
    return abs(math.cos(x)) ** 0.5 + math.sin(2 + 3 * x)

Second, as our benchmark case a pure Python implementation.

In [ ]:
I = 100000
a_py = range(I)

In [ ]:
def f1(a):
    res = []
    for x in a:
        res.append(f(x))
    return res

One can also use different paradigms to implement the same function, like **list comprehension** or the **eval** function.

In [ ]:
def f2(a):
    return [f(x) for x in a]

In [ ]:
x = 10

In [ ]:
ex = 'abs(math.cos(x)) ** 0.5 + math.sin(2 + 3 * x)'

In [ ]:
eval(ex)

In [ ]:
def f3(a):
    ex = 'abs(math.cos(x)) ** 0.5 + math.sin(2 + 3 * x)'
    return [eval(ex) for x in a]

Third, a **vectorized implementations** with NumPy.

In [ ]:
import numpy as np

In [ ]:
a_np = np.arange(I)

In [ ]:
def f4(a):
    return (np.abs(np.cos(a)) ** 0.5 +
            np.sin(2 + 3 * a))

Fourth, we use **numexpr** to evaluate the numerical expression.

In [ ]:
import numexpr as ne

In [ ]:
def f5(a):
    ex = 'abs(cos(a)) ** 0.5 + sin(2 + 3 * a)'
    ne.set_num_threads(1)
    return ne.evaluate(ex)

In [ ]:
def f6(a):
    ex = 'abs(cos(a)) ** 0.5 + sin(2 + 3 * a)'
    ne.set_num_threads(4)
    return ne.evaluate(ex)

Now, let's **compare the performance** of all different function implementations.

In [ ]:
func_list = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6']
data_list = ['a_py', 'a_py', 'a_py', 'a_np', 'a_np', 'a_np']

In [ ]:
%time perf_comp_data(func_list, data_list, rep=3)

<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>